## Analyze elongation time and collision frequencies

In [1]:
import cPickle as pkl
import pandas as pd
import numpy as np

In [2]:
files = [
'/home/martin/git/TRSL/results/1000 ribosomes, 30061 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_1137_1000_ribosomes_1200s.p',
'/home/martin/git/TRSL/results/1000 ribosomes, 60000 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_1027_1000_ribosomes_1200s.p',
'/home/martin/git/TRSL/results/2000 ribosomes, 30061 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_1502_2000_ribosomes_1200s.p',
'/home/martin/git/TRSL/results/2000 ribosomes, 60000 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_1341_2000_ribosomes_1200s.p',
'/home/martin/git/TRSL/results/5000 ribosomes, 30061 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_1847_5000_ribosomes_1200s.p',
'/home/martin/git/TRSL/results/5000 ribosomes, 60000 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_1721_5000_ribosomes_1200s.p',
'/home/martin/git/TRSL/results/10000 ribosomes, 30061 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180404_0214_10000_ribosomes_1200s.p',
'/home/martin/git/TRSL/results/10000 ribosomes, 60000 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_2216_10000_ribosomes_1200s.p',
'/home/martin/git/TRSL/results/20000 ribosomes, 30061 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180404_2134_20000_ribosomes_1200s.p',
'/home/martin/git/TRSL/results/20000 ribosomes, 60000 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180404_1135_20000_ribosomes_1200s.p',
'/home/martin/git/TRSL/results/50000 ribosomes, 30061 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180406_1349_50000_ribosomes_1200s.p',
'/home/martin/git/TRSL/results/50000 ribosomes, 60000 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180405_2214_50000_ribosomes_1200s.p',
'/home/martin/git/TRSL/results/100000 ribosomes, 30061 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180407_2239_100000_ribosomes_1200s.p',
'/home/martin/git/TRSL/results/100000 ribosomes, 60000 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180407_0928_100000_ribosomes_1200s.p',
'/home/martin/git/TRSL/results/200000 ribosomes, 30061 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180410_0108_200000_ribosomes_1200s.p',
'/home/martin/git/TRSL/results/200000 ribosomes, 60000 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180409_0404_200000_ribosomes_1200s.p',
'/home/martin/git/TRSL/results/500000 ribosomes, 30061 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180414_0349_500000_ribosomes_1200s.p',
'/home/martin/git/TRSL/results/500000 ribosomes, 60000 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180412_1054_500000_ribosomes_1200s.p',
]

To convert from numpypy to numpy:

In [3]:
from tempfile import mkstemp
from shutil import move
from os import remove, close

def replace(file_path, pattern, subst):
    #Create temp file
    fh, abs_path = mkstemp()
    with open(abs_path,'w') as new_file:
        with open(file_path) as old_file:
            for line in old_file:
                new_file.write(line.replace(pattern, subst))
    close(fh)
    #Remove original file
    remove(file_path)
    #Move new file
    move(abs_path, file_path)

Run only once per file:

In [4]:
for filename in files[-2:]:
    print filename
    replace(filename, "_numpypy.multiarray", "numpy.core.multiarray")

/home/martin/git/TRSL/results/500000 ribosomes, 30061 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180414_0349_500000_ribosomes_1200s.p
/home/martin/git/TRSL/results/500000 ribosomes, 60000 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180412_1054_500000_ribosomes_1200s.p


Blacklisted species:

In [9]:
blacklist = ['ribos._bound', 'ribos._free', 'tRNA_free', 'tRNA_bound', 'ATP', 'GTP', 'AMP', 'GDP', 'protein', 
             'proteins', 'peptide_bonds']

Data frame at cell level:

In [10]:
dfc = pd.DataFrame(columns=['ribosomes', 'transcripts', 'collisions', 'nocollisions', 'peptide_bonds'])

Fill data frames:

In [11]:
rows = []
dft_columns = ['length', 'init_rate', 'ribosomes', 'tic_toc', 'geneID', 'transcripts', 'index']

for i, filename in enumerate(files):
    print filename
    results = pkl.load(open(filename))
    
    # append to cell level data frame
    print 'cell level...'
    rowvals = [results['n_ribosomes'], len(results['transcriptome']), results['collisions'], 
               results['nocollisions'], results['peptide_bonds']]
    row = dict(zip(list(dfc.columns), rowvals))
    #print row
    dfc = dfc.append(row, ignore_index=True)
    #print dfc.head()
    
    # append to transcript level data frame
    print 'transcript level...'
    
    for transcript in results['transcriptome']:
        av_time = np.mean([tictoc[1] - tictoc[0] for tictoc in transcript.tic_toc])
        rowvals = [transcript.geneID, transcript.index, transcript.init_rate, transcript.length, 
                   results['n_ribosomes'], av_time, len(results['transcriptome'])]
        row = dict(zip(list(dft_columns), rowvals))
        #print row
        rows.append(row)
            
    # append to gene level data frame
    print 'gene level...'
    header = str(results['n_ribosomes']) + '_' + str(len(results['transcriptome']))
    tc = results['timecourses']
    efficiency_items = [(gene, (tc[gene][-1] - tc[gene][-6001]) / 300.) for gene in tc if gene not in blacklist]
    tmp = pd.DataFrame(efficiency_items, columns = ['gene', header]).set_index('gene')
    
    if i == 0:  # first time
        dfg = tmp
    else:
        dfg = pd.merge(dfg, tmp, left_index=True, right_index=True, how='outer')

dft = pd.DataFrame(rows)
dft.columns = dft_columns

/home/martin/git/TRSL/results/1000 ribosomes, 30061 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_1137_1000_ribosomes_1200s.p
cell level...
transcript level...
gene level...
/home/martin/git/TRSL/results/1000 ribosomes, 60000 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_1027_1000_ribosomes_1200s.p
cell level...
transcript level...
gene level...
/home/martin/git/TRSL/results/2000 ribosomes, 30061 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_1502_2000_ribosomes_1200s.p
cell level...
transcript level...
gene level...
/home/martin/git/TRSL/results/2000 ribosomes, 60000 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_1341_2000_ribosomes_1200s.p
cell lev

In [12]:
dfc

,ribosomes,transcripts,collisions,nocollisions,peptide_bonds
0,1000.0,28808.0,37.0,31316.0,8.984675e+06
1,1000.0,57558.0,24.0,35343.0,1.006811e+07
2,2000.0,28808.0,120.0,62677.0,1.798140e+07
3,2000.0,57558.0,83.0,70186.0,2.009996e+07
4,5000.0,28808.0,850.0,158733.0,4.513770e+07
5,5000.0,57558.0,559.0,175911.0,5.060733e+07
6,10000.0,28808.0,3486.0,321005.0,9.157788e+07
7,10000.0,57558.0,2334.0,354206.0,1.021989e+08
8,20000.0,28808.0,14512.0,654681.0,1.871142e+08
9,20000.0,57558.0,9326.0,719470.0,2.075538e+08


In [19]:
dft.columns = ['ribosomes', 'transcripts', 'index', 'geneID', 'init_rate', 'length', 'tic_toc']

In [20]:
dft

,ribosomes,transcripts,index,geneID,init_rate,length,tic_toc
0,1000,28808,0,YBR177C,1.644214e-06,1356,NaN
1,1000,28808,1,YBR177C,1.644214e-06,1356,NaN
2,1000,28808,2,YBR177C,1.644214e-06,1356,71.750000
3,1000,28808,3,YBR177C,1.644214e-06,1356,NaN
4,1000,28808,4,YIL140W,1.649229e-06,2472,124.900000
5,1000,28808,5,YIL140W,1.649229e-06,2472,120.450000
6,1000,28808,6,YLR268W,1.844285e-06,645,42.100000
7,1000,28808,7,YLR268W,1.844285e-06,645,NaN
8,1000,28808,8,YLR268W,1.844285e-06,645,NaN
9,1000,28808,9,YLR268W,1.844285e-06,645,NaN


In [15]:
dfg

,1000_28808,1000_57558,2000_28808,2000_57558,5000_28808,5000_57558,10000_28808,10000_57558,20000_28808,20000_57558,50000_28808,50000_57558,100000_28808,100000_57558,200000_28808,200000_57558,500000_28808,500000_57558
gene,,,,,,,,,,,,,,,,,,
YAL001C,NaN,NaN,0.000000,NaN,0.000000,0.003333,0.006667,0.010000,0.013333,0.010000,0.003333,0.023333,0.043333,0.043333,0.106667,0.116667,0.250000,0.326667
YAL002W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000,NaN,0.006667,NaN,0.013333,NaN,0.023333
YAL003W,0.180000,0.276667,0.480000,0.516667,1.193333,1.100000,2.403333,2.536667,4.826667,4.986667,12.563333,13.286667,25.736667,28.446667,51.910000,58.710000,115.633333,147.100000
YAL007C,0.000000,0.003333,0.006667,0.006667,0.006667,0.026667,0.030000,0.033333,0.020000,0.053333,0.173333,0.213333,0.260000,0.376667,0.613333,0.746667,1.446667,2.216667
YAL008W,NaN,NaN,NaN,NaN,0.003333,0.003333,0.010000,0.006667,0.006667,0.033333,0.050000,0.063333,0.083333,0.073333,0.173333,0.203333,0.346667,0.573333
YAL009W,0.003333,NaN,NaN,NaN,0.003333,NaN,0.000000,0.006667,0.016667,0.013333,0.030000,0.033333,0.050000,0.046667,0.090000,0.093333,0.266667,0.290000
YAL010C,NaN,NaN,0.003333,0.003333,0.003333,0.003333,0.000000,0.010000,0.003333,0.000000,0.010000,0.016667,0.016667,0.040000,0.046667,0.076667,0.166667,0.206667
YAL011W,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.003333,0.003333,0.016667,0.006667,0.026667,0.036667,0.060000,0.066667,0.153333,0.146667,0.353333,0.373333
YAL012W,0.036667,0.040000,0.053333,0.066667,0.160000,0.150000,0.283333,0.326667,0.653333,0.610000,1.660000,1.620000,3.183333,3.680000,6.916667,7.580000,12.893333,19.386667


In [16]:
dfc.to_pickle('../../results/dfc.p')

In [21]:
dft.to_pickle('../../results/dft.p')

In [18]:
dfg.to_pickle('../../results/dfg.p')